# 랩 5: 고객 지원 에이전트를 위한 GenAI 관찰 가능성 심층 분석

## 개요

이 랩에서는 AgentCore Observability가 어떻게 작동하는지, 그리고 AgentCore Runtime을 사용하지 않고 설정하는 방법을 이해하게 됩니다.

## 추가할 내용

🧠 **AgentCore 관찰 가능성 기능**:
- Amazon OpenTelemetry Python Instrumentation **설정**
- Amazon CloudWatch GenAI Observability에서 에이전트 추적 **시각화 및 분석**

## 튜토리얼 세부 사항

| 정보 | 세부 사항 |
|-------------|---------|
| **튜토리얼 유형** | 점진적 향상 |
| **에이전트 유형** | 단일 에이전트 |
| **에이전트 프레임워크** | Strands Agents |
| **LLM 모델** | Anthropic Claude 3.7 Sonnet |
| **튜토리얼 분야** | 고객 지원 |
| **복잡도** | 쉽음에서 보통 |
| **사용된 SDK** | Strands SDK, AgentCore 관찰 가능성, Cloudwatch, Bedrock, boto3 |

## 전제조건

- ✅ **랩 1을 먼저 완료해야 함** - 이 랩은 랩 1 에이전트를 직접 기반으로 구축됩니다
- ✅ **Amazon CloudWatch에서 트랜잭션 검색 활성화** - 처음 사용자는 Bedrock AgentCore 스팬과 추적을 보기 위해 CloudWatch Transaction Search를 활성화해야 합니다. 트랜잭션 검색을 활성화하려면 [문서](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html)를 참조하세요.

## 학습 목표

이 랩을 마치면 다음을 할 수 있습니다:
- 공식 Amazon CloudWatch GenAI 관찰 가능성 대시보드 사용


---

## 🚀 에이전트에 관찰 가능성 추가하기


클라이언트 초기화

In [ ]:
import boto3
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

logs_client = boto3.client("logs", region_name=region)
bedrock_client = boto3.client("bedrock", region_name=region)
sts_client = boto3.client("sts", region_name=region)

account_id = sts_client.get_caller_identity()["Account"]

`aws-opentelemetry-distro`가 설치되어 있는지 확인하세요

In [ ]:
%pip install strands-agents boto3 aws-opentelemetry-distro -q

# 2단계: 관찰 가능성을 위한 환경 구성

Strands 에이전트에 대한 관찰 가능성을 활성화하고 원격 측정 데이터를 Amazon CloudWatch로 전송하려면 다음 환경 변수를 구성해야 합니다. 민감한 AWS 자격 증명을 코드와 별도로 유지하면서 다른 환경 간 전환을 쉽게 하기 위해 이러한 설정을 안전하게 관리할 `.env` 파일을 만들겠습니다.

필수 환경 변수:

| 변수 | 값 | 목적 |
|----------|-------|---------|
| `OTEL_PYTHON_DISTRO` | `aws_distro` | OpenTelemetry용 AWS Distro (ADOT) 사용 |
| `OTEL_PYTHON_CONFIGURATOR` | `aws_configurator` | ADOT SDK용 AWS 구성자 설정 |
| `OTEL_EXPORTER_OTLP_PROTOCOL` | `http/protobuf` | 내보내기 프로토콜 구성 |
| `OTEL_TRACES_EXPORTER` | `otlp` | 추적 내보내기 구성 |
| `OTEL_EXPORTER_OTLP_LOGS_HEADERS` | `x-aws-log-group=<YOUR-LOG-GROUP>,x-aws-log-stream=<YOUR-LOG-STREAM>,x-aws-metric-namespace=<YOUR-NAMESPACE>` | CloudWatch 그룹으로 로그 직접 전송 |
| `OTEL_RESOURCE_ATTRIBUTES` | `service.name=<YOUR-에이전트-NAME>` | 관찰 가능성 데이터에서 에이전트 식별 |
| `AGENT_OBSERVABILITY_ENABLED` | `true` | ADOT 파이프라인 활성화 |

또한 `AWS_REGION`, `AWS_DEFAULT_REGION`, `AWS_ACCOUNT_ID` 환경 변수를 설정해야 합니다. 이들은 opentelemetry instrument 스크립트에서 사용됩니다.

In [ ]:
log_group_name = "agents/customer-support-assistant-logs"  # Your log group name
log_stream_name = "default"  # Your log stream name

# Create log group
try:
    logs_client.create_log_group(logGroupName=log_group_name)
    print(f"✅ Log group '{log_group_name}' created successfully")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceAlreadyExistsException":
        print(f"ℹ️  Log group '{log_group_name}' already exists")
    else:
        print(f"❌ Error creating log group: {e}")

# Create log stream
try:
    logs_client.create_log_stream(
        logGroupName=log_group_name, logStreamName=log_stream_name
    )
    print(f"✅ Log stream '{log_stream_name}' created successfully")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceAlreadyExistsException":
        print(f"ℹ️  Log stream '{log_stream_name}' already exists")
    else:
        print(f"❌ Error creating log stream: {e}")

In [ ]:
# Create .env file
service_name = "customer-support-assistant-strands"

with open(".env", "w") as f:
    # AWS Configuration
    f.write(f"AWS_REGION={region}\n")
    f.write(f"AWS_DEFAULT_REGION={region}\n")
    f.write(f"AWS_ACCOUNT_ID={account_id}\n")

    # OpenTelemetry Configuration for AWS CloudWatch GenAI Observability
    f.write("OTEL_PYTHON_DISTRO=aws_distro\n")
    f.write("OTEL_PYTHON_CONFIGURATOR=aws_configurator\n")
    f.write("OTEL_EXPORTER_OTLP_PROTOCOL=http/protobuf\n")
    f.write("OTEL_TRACES_EXPORTER=otlp\n")
    f.write(
        f"OTEL_EXPORTER_OTLP_LOGS_HEADERS=x-aws-log-group={log_group_name},x-aws-log-stream={log_stream_name},x-aws-metric-namespace=agents\n"
    )
    f.write(f"OTEL_RESOURCE_ATTRIBUTES=service.name={service_name}\n")
    f.write("AGENT_OBSERVABILITY_ENABLED=true\n")

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Display the OTEL-related environment variables
otel_vars = [
    "OTEL_PYTHON_DISTRO",
    "OTEL_PYTHON_CONFIGURATOR",
    "OTEL_EXPORTER_OTLP_PROTOCOL",
    "OTEL_EXPORTER_OTLP_LOGS_HEADERS",
    "OTEL_RESOURCE_ATTRIBUTES",
    "AGENT_OBSERVABILITY_ENABLED",
    "OTEL_TRACES_EXPORTER",
]

print("OpenTelemetry Configuration:\n")
for var in otel_vars:
    value = os.getenv(var)
    if value:
        print(f"{var}={value}")

# 3단계: Strands 에이전트 정의

이제 이전과 동일한 에이전트를 다시 정의해보겠습니다.

추적이 생성되는지 보여주기 위해 에이전트에 간단한 인사 쿼리를 전달하겠습니다.

또한 세션 ID가 등록되도록 하겠습니다.

In [ ]:
!cp lab_helpers/lab1_strands_agent.py customer_support_agent.py

In [ ]:
%%writefile -a customer_support_agent.py

import os
import argparse
from boto3.session import Session
from opentelemetry import baggage, context
from scripts.utils import get_ssm_parameter

from strands import Agent
from strands.models import BedrockModel


def parse_arguments():
    parser = argparse.ArgumentParser(description="Customer Support Agent")
    parser.add_argument(
        "--session-id",
        type=str,
        required=True,
        help="Session ID to associate with this agent run",
    )
    return parser.parse_args()


def set_session_context(session_id):
    """Set the session ID in OpenTelemetry baggage for trace correlation"""
    ctx = baggage.set_baggage("session.id", session_id)
    token = context.attach(ctx)
    print(f"Session ID '{session_id}' attached to telemetry context")
    return token


def main():
    # Parse command line arguments
    args = parse_arguments()

    # Set session context for telemetry
    context_token = set_session_context(args.session_id)

    # Get region
    boto_session = Session()
    region = boto_session.region_name

    try:
        # Create the same basic agent from Lab 1
        MODEL = BedrockModel(
            model_id=MODEL_ID,
            temperature=0.3,
            region_name=region,
        )

        basic_agent = Agent(
            model=MODEL,
            tools=[
                get_product_info,
                get_return_policy,
            ],
            system_prompt=SYSTEM_PROMPT,
        )

        # Execute the travel research task
        query = """Greet the user and provide a financial advice."""

        result = basic_agent(query)
        print("Result:", result)

        print("✅ Agent executed successfully and trace was pushed to CloudWatch")
    finally:
        # Detach context when done
        context.detach(context_token)


if __name__ == "__main__":
    main()

# 4단계: AWS OpenTelemetry Python Distro

이제 환경이 구성되고 에이전트가 생성되었으므로 관찰 가능성이 어떻게 작동하는지 이해해보겠습니다. [AWS OpenTelemetry Python Distro](https://pypi.org/project/aws-opentelemetry-distro/)는 코드 변경 없이 Strands 에이전트를 자동으로 계측하여 원격 측정 데이터를 캐처합니다.

이 배포판은 다음을 제공합니다:
- AgentCore 런타임 외부에서 호스팅되는 Strands 에이전트에 대한 **자동 계측** (EC2, Lambda 등)
- 원활한 CloudWatch 통합을 위한 **AWS 최적화 구성**

### 계측된 에이전트 실행

Strands 에이전트에서 추적을 캐처하려면 Python을 직접 실행하는 대신 `opentelemetry-instrument` 명령을 사용하세요. 이는 `.env` 파일의 환경 변수를 사용하여 자동으로 계측을 적용합니다:

```bash
opentelemetry-instrument python customer_support_assistant_agent.py
```

이 명령은 다음을 수행합니다:

- .env 파일에서 OTEL 구성 로드
- Strands, Amazon Bedrock 호출, 에이전트 도구 및 데이터베이스, 에이전트가 수행하는 기타 요청 자동 계측
- CloudWatch로 추적 전송
- GenAI 관찰 가능성 대시보드에서 에이전트의 의사결정 과정 시각화 가능

In [ ]:
!opentelemetry-instrument python customer_support_agent.py --session-id "session-1234"

# 5단계: Gen AI Observability에서 보기

이제 Observability를 구성했으므로 AWS CloudWatch의 GenAI 관찰 가능성 대시보드에서 추적을 확인해보겠습니다. Cloudwatch - GenAI 관찰 가능성 - Bedrock AgentCore로 이동하세요.

#### 세션 보기 페이지:

![sessions](images/sessions_lab5_observability.png)

#### 추적 보기 페이지:
![traces](images/traces_lab5_observability.png)


## 축하합니다! 🎉

**Strands 에이전트로 AgentCore Observability를 성공적으로 구현**했습니다 (AgentCore 런타임 없이)!

### 달성한 내용:

- ✅ **관찰 가능성**: Strands 에이전트가 Amazon CloudWatch로 원격 측정 데이터를 전송하도록 구성
- ✅ **세션 관리**: 더 쉽운 디버깅을 위해 추적이 세션별로 저장되도록 보장

## 다음 단계

더 많은 AgentCore 기능을 추가할 준비가 되셨나요? 다음으로 계속하세요:

- **랩 6**: AgentCore Identity를 사용하여 외부 서비스와 안전하게 인증

## 리소스

- [AgentCore 관찰 가능성 문서](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/관찰 가능성.html)
- [**공식 AgentCore 관찰 가능성 샘플**](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/01-tutorials/06-AgentCore-관찰 가능성) ⭐

---

**훌륭한 작업입니다! 이제 프로덕션 환경에서 고객 지원 에이전트의 성능을 추적, 디버깅, 모니터링할 수 있습니다! 🚀**
